# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses using the regression methods for dependent data that we are covering in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here". Then enter your responses to the questions in the boxes labeled "Type Markdown and Latex".

This notebook is based on the NHANES case study notebook for regression with dependent data.  Most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "BPXDI1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model for the first measurement of diastolic blood pressure (`BPXDI1`), using GEE to account for the grouping variable 'group' defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [19]:
m0 = sm.GEE.from_formula("BPXDI1 ~ 1", "group", cov_struct=sm.cov_struct.Exchangeable(), data=da)
r0 = m0.fit()
r0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               GEE Regression Results                              
===================================================================================
Dep. Variable:                      BPXDI1   No. Observations:                 5102
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                 106
                      Estimating Equations   Max. cluster size:                 226
Family:                           Gaussian   Mean cluster size:               170.1
Dependence structure:         Exchangeable   Num. iterations:                     5
Date:                     Wed, 06 Sep 2023   Scale:                         162.315
Covariance type:                    robust   Time:                         16:45:29
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     70.0088      0.419    167.219      0.000      69.188      70.829
==============================================================================
Skew:                         -0.6467   Kurtosis:                       3.8275
Centered skew:                -0.6299   Centered kurtosis:              3.8513
==============================================================================
"""

__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

In [20]:
m1 = sm.GEE.from_formula("BPXSY1 ~ 1", "group", cov_struct=sm.cov_struct.Exchangeable(), data=da)
r1 = m1.fit()
print(r0.cov_struct.summary())
print(r1.cov_struct.summary())

The correlation between two observations in the same cluster is 0.031
The correlation between two observations in the same cluster is 0.030


The ICC for diastolic and systolic blood pressure are quite similar.  For both diastolic and systolic blood pressure, around 3% of the variance is between groups and 97% is within groups.

## Question 2: 

Take your model from question 1, and add sex, age, and BMI to it as covariates.

In [21]:
m2 = sm.GEE.from_formula("BPXDI1 ~ RIAGENDR + RIDAGEYR + BMXBMI", "group", cov_struct=sm.cov_struct.Exchangeable(), data=da)
r2 = m2.fit()
r2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               GEE Regression Results                              
===================================================================================
Dep. Variable:                      BPXDI1   No. Observations:                 5102
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                 106
                      Estimating Equations   Max. cluster size:                 226
Family:                           Gaussian   Mean cluster size:               170.1
Dependence structure:         Exchangeable   Num. iterations:                     7
Date:                     Wed, 06 Sep 2023   Scale:                         158.611
Covariance type:                    robust   Time:                         16:45:30
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     70.7858      1.098     64.489      0.000      68.634      72.937
RIAGENDR      -2.7422      0.402     -6.828      0.000      -3.529      -1.955
RIDAGEYR      -0.0409      0.014     -2.845      0.004      -0.069      -0.013
BMXBMI         0.1839      0.030      6.218      0.000       0.126       0.242
==============================================================================
Skew:                         -0.6865   Kurtosis:                       3.9310
Centered skew:                -0.6664   Centered kurtosis:              3.9656
==============================================================================
"""

__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

In [22]:
r1.cov_struct.summary()

'The correlation between two observations in the same cluster is 0.030'

The ICC is almost identical to what we saw with the unadjusted model above.  Evidently the covariates (sex, age, BMI) do not account for the ICC.  This will happen when the covariates are approximately equally distributed across all levels of 'group', and/or when the covariates do not explain any variation in the response variable (either diastolic or systolic blood pressure here).

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [23]:
for ky,dz in da.groupby("RIAGENDR"):
    m2 = sm.GEE.from_formula("BPXDI1 ~ RIDAGEYR + BMXBMI", "group", cov_struct=sm.cov_struct.Exchangeable(), data=dz)
    r2 = m2.fit()
    print("Sex=%s" % {1: "Male", 2: "Female"}[ky])
    print(r2.summary())

Sex=Male
                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 2462
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                  38
                      Estimating Equations   Max. cluster size:                 111
Family:                           Gaussian   Mean cluster size:                82.1
Dependence structure:         Exchangeable   Num. iterations:                     7
Date:                     Wed, 06 Sep 2023   Scale:                         173.641
Covariance type:                    robust   Time:                         16:45:30
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     66.8949      1.751     38.210      0.000      63.464      70.32

__Q3a.__ What do you learn by comparing these two fitted models?

BMI is a statistically significant predictor of diastolic blood pressure for both sexes.  The quantitative association between BMI and blood pressure is stronger for males than for females.  Age is not a statistically significant predictor of diastolic blood pressure for females, and is negatively associated with diastolic blood pressure for males.

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [24]:
m4 = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDR", groups="group", data=da)
r4 = m4.fit()
r4.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:            MixedLM Dependent Variable: BPXDI1     
No. Observations: 5102    Method:             REML       
No. Groups:       30      Scale:              155.7897   
Min. group size:  106     Log-Likelihood:     -20145.7435
Max. group size:  226     Converged:          Yes        
Mean group size:  170.1                                  
----------------------------------------------------------
           Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept  75.852     0.843  90.006  0.000  74.200  77.504
RIDAGEYR   -0.040     0.010  -3.997  0.000  -0.060  -0.021
RIAGENDR   -2.545     0.351  -7.258  0.000  -3.232  -1.857
group Var   4.082     0.105                               
=========================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

In [25]:
4/da["BPXDI1"].var()

0.024645775049710904

The ICC from the mixed model is `4/da["BPXDI1"].var()`, which is similar to the ICC found above using GEE.  According to the multilevel regression model, around 2.5% of the diastolic blood pressure variance is between-groups and 98.5% is within groups.

__Q4b:__ Include a random intercept for age, and describe your findings.

The multilevel model fitting algorithm may not converge especially when including random slopes for variables with high mean and/or variance.  Therefore the age variable is centered and scaled below before incorporating a random slope into the model.  Also, we choose to model the random intercepts and slopes as independent random variables.

This model indicates that there is a negligible random slope for age.  The convergence warning is expected when an estimated variance parameter is nearly equal to zero.  Thus, there is little evidence that different groups have different age slopes.

In [26]:
da["RIDAGEYR_cen"] = (da["RIDAGEYR"] - da["RIDAGEYR"].mean()) / 10
m5 = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDR", groups="group", re_formula="1", 
                              vc_formula={"RIDAGEYR_cen": "0+RIDAGEYR"}, data=da)
r5 = m5.fit(method="lbfgs")
r5.summary()

/nfs/turbo/lsa-kshedden/statsmodels/statsmodels/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
===========================================================
Model:              MixedLM Dependent Variable: BPXDI1     
No. Observations:   5102    Method:             REML       
No. Groups:         30      Scale:              154.8623   
Min. group size:    106     Log-Likelihood:     -20144.4657
Max. group size:    226     Converged:          Yes        
Mean group size:    170.1                                  
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        75.648    0.845 89.537 0.000 73.992 77.304
RIDAGEYR         -0.035    0.013 -2.671 0.008 -0.061 -0.009
RIAGENDR         -2.522    0.350 -7.210 0.000 -3.208 -1.837
group Var         4.041    0.254                           
RIDAGEYR_cen Var  0.002    0.000                           
===========================================================

"""